<a href="https://colab.research.google.com/github/waqasm86/Kaggle-Dropbox-HuggingFace/blob/main/p2_colab_dropbox_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#cpu compute
!pip -q install -U huggingface_hub dropbox requests > /dev/null

In [15]:
#gpu compute
!pip -q install -U  tqdm requests transformers accelerate bitsandbytes sentencepiece einops safetensors openai >/dev/null

In [16]:
import torch, os, json, pathlib, gc

In [18]:
import os, json, pathlib, posixpath, re, sys, time
from typing import Optional, Iterable
import dropbox
from dropbox.exceptions import AuthError, ApiError

In [19]:
DROPBOX_FOLDER = "/p1-qwen2.5-3b-Instruct"   # your Dropbox path from previous step
LOCAL_ROOT     = pathlib.Path("/content/qwen2.5-3b-Instruct")
LOCAL_ROOT.mkdir(parents=True, exist_ok=True)

INCLUDE_SUFFIXES = {
    ".safetensors", ".index.json",
    ".json", ".md", ".gitattributes", ".txt", ".model", ".vocab", ".merges"
}
# files we definitely want if present
MUST_HAVE = {
    "config.json",
    "generation_config.json",
    "tokenizer.json",
    "tokenizer_config.json",
    "merges.txt",
    "vocab.json",
    "model.safetensors.index.json"
}

In [ ]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="sk-lf-194ee11a-d986-41e0-b6d7-6b4e0415291e",
  public_key="pk-lf-186192e8-9f2b-41ad-be1e-109efad2d3a3",
  host="https://us.cloud.langfuse.com"
)

In [17]:
import os, pathlib
from google.colab import userdata

# ---- Dropbox (preferred: offline/refreshable) ----
os.environ["DBX_APP_KEY"]       = userdata.get('DBX_APP_KEY')
os.environ["DBX_APP_SECRET"]    = userdata.get('DBX_APP_SECRET')
os.environ["DBX_REFRESH_TOKEN"] = userdata.get('DBX_REFRESH_TOKEN')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

DBX_MODEL_DIR  = "/p1-qwen2.5-3b-Instruct"         # your Dropbox folder from Kaggle step
LOCAL_MODEL_DIR = "/content/qwen2.5-3b-Instruct"   # Colab temp storage
pathlib.Path(LOCAL_MODEL_DIR).mkdir(parents=True, exist_ok=True)

In [5]:
import dropbox
from dropbox.exceptions import AuthError

def dropbox_connect():
    try:
        dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
        dbx.users_get_current_account()  # Test connection
        print("Successfully connected to Dropbox!")
        return dbx
    except AuthError as e:
        print("Error connecting to Dropbox:", e)
        return None

dbx = dropbox_connect()

Error connecting to Dropbox: AuthError('762fbfebecda469d9a383bea364445f7', AuthError('expired_access_token', None))


In [ ]:
def upload_to_dropbox(local_path, dropbox_path):
    dbx = dropbox.Dropbox(
        DROPBOX_ACCESS_TOKEN,
        timeout=30  # Prevents timeout errors
    )
    try:
        for root, dirs, files in os.walk(local_path):
            for file in files:
                local_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(local_file_path, local_path)
                dropbox_file_path = os.path.join(dropbox_path, relative_path).replace("\\", "/")  # Fix path format

                # Chunked upload for large files (avoids memory issues)
                CHUNK_SIZE = 4 * 1024 * 1024  # 4MB chunks
                with open(local_file_path, "rb") as f:
                    file_size = os.path.getsize(local_file_path)
                    if file_size <= CHUNK_SIZE:
                        dbx.files_upload(f.read(), dropbox_file_path, mode=dropbox.files.WriteMode("overwrite"))
                    else:
                        upload_session_start_result = dbx.files_upload_session_start(f.read(CHUNK_SIZE))
                        cursor = dropbox.files.UploadSessionCursor(
                            session_id=upload_session_start_result.session_id,
                            offset=f.tell()
                        )
                        while f.tell() < file_size:
                            remaining = file_size - f.tell()
                            chunk = f.read(min(CHUNK_SIZE, remaining))
                            dbx.files_upload_session_append_v2(chunk, cursor)
                            cursor.offset = f.tell()
                        dbx.files_upload_session_finish(None, cursor, dropbox.files.CommitInfo(path=dropbox_file_path))
                print(f"Uploaded: {dropbox_file_path}")
    except Exception as e:
        print(f"Error uploading to Dropbox: {e}")
        raise

In [3]:
# %%colab
# HF -> Dropbox (Colab version; no temp files on disk)
# - Secrets: taken from google.colab.userdata when available
#            else parsed from /content/Colab-Credentials.txt (KEY on one line, VALUE on next)
# - Preferred Dropbox auth: refresh token (offline access)
# - Progress: one clean tqdm bar per large file

import os, sys, io, posixpath, time, requests, textwrap
from typing import Optional, Dict
from contextlib import redirect_stderr
from tqdm import tqdm

!pip -q install --upgrade huggingface_hub dropbox requests

from huggingface_hub import HfApi, hf_hub_url
import dropbox
from dropbox.exceptions import ApiError, AuthError

# =========================
# 1) Configuration
# =========================
MODEL_ID       = "Qwen/Qwen2.5-3B-Instruct"
DROPBOX_FOLDER = "/p1-qwen2.5-3b-Instruct"   # must start with "/"
CHUNK          = 8 * 1024 * 1024             # 8 MB for Dropbox upload sessions
MAX_RETRIES    = 5
TIMEOUT        = (10, 120)                   # (connect, read) seconds
SHOW_BAR_MIN   = 1 * 1024 * 1024             # show tqdm only for >= 1MB files

INCLUDE_SUFFIXES = {".safetensors", ".json", ".md", ".txt", ".gitattributes", ".jinja"}
EXCLUDE_PREFIXES = ("original/", "metal/")

CREDENTIALS_TXT = "/content/Colab-Credentials.txt"  # optional fallback

# =========================
# 2) Secrets loader
# =========================
def _load_from_user_data(key: str) -> Optional[str]:
    try:
        from google.colab import userdata  # type: ignore
        v = userdata.get(key)
        if isinstance(v, str):
            v = v.strip()
        return v or None
    except Exception:
        return None

def _parse_credentials_txt(path: str) -> Dict[str, str]:
    if not os.path.exists(path):
        return {}
    # Expect alternating KEY \n VALUE \n blocks
    data: Dict[str, str] = {}
    with open(path, "r", encoding="utf-8") as f:
        lines = [ln.rstrip("\n") for ln in f]
    i = 0
    while i < len(lines):
        key = lines[i].strip()
        val = ""
        if i + 1 < len(lines):
            val = lines[i + 1].strip()
        if key and val and not key.startswith("#"):
            data[key] = val
        i += 2
    return data

def get_secret(name: str) -> Optional[str]:
    v = _load_from_user_data(name)
    if v:
        return v
    env = os.environ.get(name)
    if env:
        return env.strip()
    txt = _parse_credentials_txt(CREDENTIALS_TXT).get(name)
    if txt:
        return txt.strip()
    return None

HF_TOKEN            = get_secret("HF_TOKEN") or get_secret("HUGGINGFACE_TOKEN")
DBX_APP_KEY         = get_secret("DBX_APP_KEY")
DBX_APP_SECRET      = get_secret("DBX_APP_SECRET")
DBX_REFRESH_TOKEN   = get_secret("DBX_REFRESH_TOKEN")
DROPBOX_ACCESS_TOKEN= get_secret("DROPBOX_ACCESS_TOKEN")

# =========================
# 3) Clients
# =========================
api = HfApi()

def get_dbx_client():
    # Prefer refresh token flow
    if DBX_APP_KEY and DBX_APP_SECRET and DBX_REFRESH_TOKEN:
        dbx = dropbox.Dropbox(
            oauth2_refresh_token=DBX_REFRESH_TOKEN,
            app_key=DBX_APP_KEY,
            app_secret=DBX_APP_SECRET,
            timeout=300
        )
    elif DROPBOX_ACCESS_TOKEN:
        dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN, timeout=300)
    else:
        raise SystemExit(
            "No Dropbox credentials found. Provide (DBX_APP_KEY, DBX_APP_SECRET, DBX_REFRESH_TOKEN) "
            "or DROPBOX_ACCESS_TOKEN via Colab userdata or Colab-Credentials.txt."
        )
    # Verify
    try:
        acc = dbx.users_get_current_account()
        print(f"✅ Dropbox: {acc.name.display_name}")
    except AuthError as e:
        raise SystemExit(f"Dropbox auth failed: {e}")
    return dbx

dbx = get_dbx_client()

# Ensure folder tree exists (idempotent)
def ensure_folder(path: str):
    path = path.rstrip("/")
    if not path or path == "/":
        return
    parts = [p for p in path.split("/") if p]
    cur = ""
    for p in parts:
        cur = "/" + (p if not cur else cur.strip("/") + "/" + p)
        try:
            dbx.files_create_folder_v2(cur)
        except ApiError:
            pass

ensure_folder(DROPBOX_FOLDER)

# =========================
# 4) Repo file selection
# =========================
def want_file(relpath: str) -> bool:
    if relpath.startswith(EXCLUDE_PREFIXES):
        return False
    return any(relpath.endswith(ext) for ext in INCLUDE_SUFFIXES)

repo_files = api.list_repo_files(MODEL_ID, repo_type="model")
wanted = [p for p in repo_files if want_file(p)]
print(f"Found {len(repo_files)} files; will transfer {len(wanted)} essential files.")

# =========================
# 5) HTTP session (HF)
# =========================
session = requests.Session()
if HF_TOKEN:
    session.headers["Authorization"] = f"Bearer {HF_TOKEN}"

def stream_from_hf(relpath: str):
    url = hf_hub_url(MODEL_ID, filename=relpath, repo_type="model", revision="main")
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = session.get(url, stream=True, timeout=TIMEOUT, allow_redirects=False)
            if r.status_code in (301, 302, 303, 307, 308):
                redir = r.headers.get("Location")
                r.close()
                r = session.get(redir, stream=True, timeout=TIMEOUT)
            r.raise_for_status()
            size = int(r.headers.get("Content-Length", "0")) if r.headers.get("Content-Length") else 0
            return r, size
        except Exception as e:
            if attempt == MAX_RETRIES:
                raise
            time.sleep(1.5 * attempt)

def file_exists_in_dropbox(dst_path: str) -> bool:
    try:
        dbx.files_get_metadata(dst_path)
        return True
    except ApiError:
        return False

def upload_stream_to_dropbox(dst_path: str, stream, total_size: int):
    ensure_folder(posixpath.dirname(dst_path))

    first = stream.raw.read(CHUNK, decode_content=True)
    if not first:
        dbx.files_upload(b"", dst_path, mode=dropbox.files.WriteMode("overwrite"))
        tqdm.write(f"✔ {posixpath.basename(dst_path)} (empty)")
        return

    start_res = dbx.files_upload_session_start(first)
    cursor = dropbox.files.UploadSessionCursor(session_id=start_res.session_id, offset=len(first))
    commit = dropbox.files.CommitInfo(path=dst_path, mode=dropbox.files.WriteMode("overwrite"))

    show_bar = (total_size or 0) >= SHOW_BAR_MIN
    pbar = None
    if show_bar:
        pbar = tqdm(
            total=total_size if total_size else None,
            unit="B", unit_scale=True, unit_divisor=1024,
            desc=posixpath.basename(dst_path),
            dynamic_ncols=True, mininterval=1.0, maxinterval=5.0, smoothing=0.1,
            leave=False, position=0, file=sys.stdout,
        )
        pbar.update(len(first))

    def bump(n):
        if pbar: pbar.update(n)

    while True:
        chunk = stream.raw.read(CHUNK, decode_content=True)
        if not chunk:
            break
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                if (total_size and (cursor.offset + len(chunk)) < total_size) or (total_size == 0):
                    dbx.files_upload_session_append_v2(chunk, cursor)
                    cursor.offset += len(chunk)
                else:
                    dbx.files_upload_session_finish(chunk, cursor, commit)
                    cursor.offset += len(chunk)
                break
            except Exception:
                if attempt == MAX_RETRIES:
                    if pbar: pbar.close()
                    raise
                time.sleep(1.5 * attempt)
        bump(len(chunk))

    # finalize (unknown or mismatched size)
    if total_size == 0 or (total_size and cursor.offset < total_size):
        dbx.files_upload_session_finish(b"", cursor, commit)

    if pbar:
        pbar.close()

    final_sz = total_size or cursor.offset
    size_str = f"{final_sz / (1024**3):.2f} GB" if final_sz >= (2**30) else f"{final_sz/1024:.0f} KB"
    tqdm.write(f"✔ {posixpath.basename(dst_path)} ({size_str})")

# =========================
# 6) Transfer loop
# =========================
errors = {}
done = 0

with redirect_stderr(open(os.devnull, "w")):  # silence stray tqdm on stderr
    for rel in wanted:
        dst = posixpath.join(DROPBOX_FOLDER, rel)
        try:
            if file_exists_in_dropbox(dst):
                print(f"↪️  Skip (exists): {dst}")
                done += 1
                continue
            stream, size = stream_from_hf(rel)
            upload_stream_to_dropbox(dst, stream, size)
            stream.close()
            done += 1
        except Exception as e:
            errors[rel] = str(e)
            print(f"❌ {rel}: {e}")

print(f"Done: {done} | Failed: {len(errors)} | Folder: {DROPBOX_FOLDER}")
if errors:
    for k, v in errors.items():
        print(f"  - {k}: {v}")


✅ Dropbox: M Waqas
Found 12 files; will transfer 11 essential files.
❌ .gitattributes: Invalid URL '/api/resolve-cache/models/Qwen/Qwen2.5-3B-Instruct/aa8e72537993ba99e69dfaafa59ed015b17504d1/.gitattributes?%2FQwen%2FQwen2.5-3B-Instruct%2Fresolve%2Fmain%2F.gitattributes=&etag=%22a6344aac8c09253b3b630fb776ae94478aa0275b%22': No scheme supplied. Perhaps you meant https:///api/resolve-cache/models/Qwen/Qwen2.5-3B-Instruct/aa8e72537993ba99e69dfaafa59ed015b17504d1/.gitattributes?%2FQwen%2FQwen2.5-3B-Instruct%2Fresolve%2Fmain%2F.gitattributes=&etag=%22a6344aac8c09253b3b630fb776ae94478aa0275b%22?
❌ README.md: Invalid URL '/api/resolve-cache/models/Qwen/Qwen2.5-3B-Instruct/aa8e72537993ba99e69dfaafa59ed015b17504d1/README.md?%2FQwen%2FQwen2.5-3B-Instruct%2Fresolve%2Fmain%2FREADME.md=&etag=%220b87f6362ee8ad33b2223afb47134264380f13fe%22': No scheme supplied. Perhaps you meant https:///api/resolve-cache/models/Qwen/Qwen2.5-3B-Instruct/aa8e72537993ba99e69dfaafa59ed015b17504d1/README.md?%2FQwen%2FQwe

In [6]:
# %%colab
# HF -> Dropbox (Colab version; fixed redirect handling for relative Location headers)

import os, sys, io, posixpath, time, requests
from typing import Optional, Dict
from contextlib import redirect_stderr
from urllib.parse import urljoin
from tqdm import tqdm

!pip -q install --upgrade huggingface_hub dropbox requests

from huggingface_hub import HfApi, hf_hub_url
import dropbox
from dropbox.exceptions import ApiError, AuthError

# ---------------- Config ----------------
MODEL_ID       = "Qwen/Qwen2.5-3B-Instruct"
DROPBOX_FOLDER = "/p1-qwen2.5-3b-Instruct"   # must start with "/"
CHUNK          = 8 * 1024 * 1024             # 8 MB
MAX_RETRIES    = 5
TIMEOUT        = (10, 120)                   # (connect, read) seconds
SHOW_BAR_MIN   = 1 * 1024 * 1024             # show tqdm only for >= 1MB files

INCLUDE_SUFFIXES = {".safetensors", ".json", ".md", ".txt", ".gitattributes", ".jinja"}
EXCLUDE_PREFIXES = ("original/", "metal/")

CREDENTIALS_TXT = "/content/Colab-Credentials.txt"  # optional fallback

# ---------------- Secrets loader ----------------
def _load_from_user_data(key: str) -> Optional[str]:
    try:
        from google.colab import userdata  # type: ignore
        v = userdata.get(key)
        if isinstance(v, str):
            v = v.strip()
        return v or None
    except Exception:
        return None

def _parse_credentials_txt(path: str) -> Dict[str, str]:
    if not os.path.exists(path):
        return {}
    data: Dict[str, str] = {}
    with open(path, "r", encoding="utf-8") as f:
        lines = [ln.rstrip("\n") for ln in f]
    i = 0
    while i < len(lines):
        key = lines[i].strip()
        val = lines[i + 1].strip() if i + 1 < len(lines) else ""
        if key and val and not key.startswith("#"):
            data[key] = val
        i += 2
    return data

def get_secret(name: str) -> Optional[str]:
    for source in (_load_from_user_data, os.environ.get, _parse_credentials_txt(CREDENTIALS_TXT).get):
        try:
            v = source(name) if source != os.environ.get else source(name)
        except TypeError:
            v = None
        if v:
            return v.strip()
    return None

HF_TOKEN              = get_secret("HF_TOKEN") or get_secret("HUGGINGFACE_TOKEN")
DBX_APP_KEY           = get_secret("DBX_APP_KEY")
DBX_APP_SECRET        = get_secret("DBX_APP_SECRET")
DBX_REFRESH_TOKEN     = get_secret("DBX_REFRESH_TOKEN")
DROPBOX_ACCESS_TOKEN  = get_secret("DROPBOX_ACCESS_TOKEN")

# ---------------- Clients ----------------
api = HfApi()

def get_dbx_client():
    if DBX_APP_KEY and DBX_APP_SECRET and DBX_REFRESH_TOKEN:
        dbx = dropbox.Dropbox(
            oauth2_refresh_token=DBX_REFRESH_TOKEN,
            app_key=DBX_APP_KEY,
            app_secret=DBX_APP_SECRET,
            timeout=300
        )
    elif DROPBOX_ACCESS_TOKEN:
        dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN, timeout=300)
    else:
        raise SystemExit(
            "No Dropbox credentials found. Provide (DBX_APP_KEY, DBX_APP_SECRET, DBX_REFRESH_TOKEN) "
            "or DROPBOX_ACCESS_TOKEN."
        )
    try:
        acc = dbx.users_get_current_account()
        print(f"✅ Dropbox: {acc.name.display_name}")
    except AuthError as e:
        raise SystemExit(f"Dropbox auth failed: {e}")
    return dbx

dbx = get_dbx_client()

def ensure_folder(path: str):
    path = path.rstrip("/")
    if not path or path == "/":
        return
    parts = [p for p in path.split("/") if p]
    cur = ""
    for p in parts:
        cur = "/" + (p if not cur else cur.strip("/") + "/" + p)
        try:
            dbx.files_create_folder_v2(cur)
        except ApiError:
            pass

ensure_folder(DROPBOX_FOLDER)

# ---------------- Select files ----------------
def want_file(relpath: str) -> bool:
    if relpath.startswith(EXCLUDE_PREFIXES):
        return False
    return any(relpath.endswith(ext) for ext in INCLUDE_SUFFIXES)

repo_files = api.list_repo_files(MODEL_ID, repo_type="model")
wanted = [p for p in repo_files if want_file(p)]
print(f"Found {len(repo_files)} files; will transfer {len(wanted)} essential files.")

# ---------------- HTTP session (HF) ----------------
session = requests.Session()
if HF_TOKEN:
    session.headers["Authorization"] = f"Bearer {HF_TOKEN}"

HF_BASE = "https://huggingface.co"

def stream_from_hf(relpath: str):
    """Return (response, total_size). Follows redirects, fixing relative Location headers."""
    url = hf_hub_url(MODEL_ID, filename=relpath, repo_type="model", revision="main")
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            # First try: allow requests to auto-follow
            r = session.get(url, stream=True, timeout=TIMEOUT, allow_redirects=True)
            # If server chose to *not* auto-follow and gave us a redirect, normalize it
            if 300 <= r.status_code < 400:
                r.close()
                redir = r.headers.get("Location", "")
                if not redir.startswith("http"):
                    redir = urljoin(HF_BASE, redir)
                r = session.get(redir, stream=True, timeout=TIMEOUT, allow_redirects=True)
            r.raise_for_status()
            size = int(r.headers.get("Content-Length", "0")) if r.headers.get("Content-Length") else 0
            return r, size
        except Exception:
            if attempt == MAX_RETRIES:
                raise
            time.sleep(1.5 * attempt)

def file_exists_in_dropbox(dst_path: str) -> bool:
    try:
        dbx.files_get_metadata(dst_path)
        return True
    except ApiError:
        return False

def upload_stream_to_dropbox(dst_path: str, stream, total_size: int):
    ensure_folder(posixpath.dirname(dst_path))

    first = stream.raw.read(CHUNK, decode_content=True)
    if not first:
        dbx.files_upload(b"", dst_path, mode=dropbox.files.WriteMode("overwrite"))
        tqdm.write(f"✔ {posixpath.basename(dst_path)} (empty)")
        return

    start_res = dbx.files_upload_session_start(first)
    cursor = dropbox.files.UploadSessionCursor(session_id=start_res.session_id, offset=len(first))
    commit = dropbox.files.CommitInfo(path=dst_path, mode=dropbox.files.WriteMode("overwrite"))

    show_bar = (total_size or 0) >= SHOW_BAR_MIN
    pbar = None
    if show_bar:
        pbar = tqdm(
            total=total_size if total_size else None,
            unit="B", unit_scale=True, unit_divisor=1024,
            desc=posixpath.basename(dst_path),
            dynamic_ncols=True, mininterval=1.0, maxinterval=5.0, smoothing=0.1,
            leave=False, position=0, file=sys.stdout,
        )
        pbar.update(len(first))

    def bump(n):
        if pbar: pbar.update(n)

    while True:
        chunk = stream.raw.read(CHUNK, decode_content=True)
        if not chunk:
            break
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                if (total_size and (cursor.offset + len(chunk)) < total_size) or (total_size == 0):
                    dbx.files_upload_session_append_v2(chunk, cursor)
                    cursor.offset += len(chunk)
                else:
                    dbx.files_upload_session_finish(chunk, cursor, commit)
                    cursor.offset += len(chunk)
                break
            except Exception:
                if attempt == MAX_RETRIES:
                    if pbar: pbar.close()
                    raise
                time.sleep(1.5 * attempt)
        bump(len(chunk))

    # finalize (unknown or mismatched size)
    if total_size == 0 or (total_size and cursor.offset < total_size):
        dbx.files_upload_session_finish(b"", cursor, commit)

    if pbar:
        pbar.close()

    final_sz = total_size or cursor.offset
    size_str = f"{final_sz / (1024**3):.2f} GB" if final_sz >= (1 << 30) else f"{final_sz/1024:.0f} KB"
    tqdm.write(f"✔ {posixpath.basename(dst_path)} ({size_str})")

# ---------------- Transfer loop ----------------
errors = {}
done = 0

with redirect_stderr(open(os.devnull, "w")):  # silence stray tqdm on stderr
    for rel in wanted:
        dst = posixpath.join(DROPBOX_FOLDER, rel)
        try:
            if file_exists_in_dropbox(dst):
                print(f"↪️  Skip (exists): {dst}")
                done += 1
                continue
            stream, size = stream_from_hf(rel)
            upload_stream_to_dropbox(dst, stream, size)
            stream.close()
            done += 1
        except Exception as e:
            errors[rel] = str(e)
            print(f"❌ {rel}: {e}")

print(f"Done: {done} | Failed: {len(errors)} | Folder: {DROPBOX_FOLDER}")
if errors:
    for k, v in errors.items():
        print(f"  - {k}: {v}")


✅ Dropbox: M Waqas
Found 12 files; will transfer 11 essential files.
✔ .gitattributes (1 KB)
✔ README.md (5 KB)
✔ config.json (1 KB)
✔ generation_config.json (0 KB)
✔ merges.txt (1633 KB)
↪️  Skip (exists): /p1-qwen2.5-3b-Instruct/model-00001-of-00002.safetensors
↪️  Skip (exists): /p1-qwen2.5-3b-Instruct/model-00002-of-00002.safetensors
✔ model.safetensors.index.json (35 KB)
✔ tokenizer.json (6867 KB)
✔ tokenizer_config.json (7 KB)
✔ vocab.json (2712 KB)
Done: 11 | Failed: 0 | Folder: /p1-qwen2.5-3b-Instruct


In [10]:
import os, json, pathlib

ROOT = pathlib.Path("/content/qwen2.5-3b-Instruct")  # your sync target
assert ROOT.exists(), f"{ROOT} does not exist. Mount/copy your Dropbox content there or change this path."

CANDIDATE_FILES = {"config.json", "tokenizer_config.json"}
REQUIRED_KEYS_IN_CONFIG = {"model_type"}  # should be "gpt_oss" for this model

def is_valid_model_dir(p: pathlib.Path) -> bool:
    cfg = p / "config.json"
    if not cfg.exists():
        return False
    try:
        j = json.loads(cfg.read_text())
    except Exception:
        return False
    # minimal sanity
    if not REQUIRED_KEYS_IN_CONFIG.issubset(j.keys()):
        return False
    # tokenizer files usually present (one of these typically exists)
    has_tok = any((p / name).exists() for name in ["tokenizer.json","tokenizer.model","vocab.json"])
    return has_tok

def find_model_dir(root: pathlib.Path) -> Optional[pathlib.Path]:
    # 1) try root
    if is_valid_model_dir(root):
        return root
    # 2) search one level deep
    for child in root.iterdir():
        if child.is_dir() and is_valid_model_dir(child):
            return child
    # 3) search repo-like layouts (e.g., snapshot dirs)
    for p in root.rglob("config.json"):
        if is_valid_model_dir(p.parent):
            return p.parent
    return None

MODEL_DIR = find_model_dir(ROOT)
print("Resolved MODEL_DIR:", MODEL_DIR)


Resolved MODEL_DIR: None


In [11]:
from huggingface_hub import snapshot_download

if MODEL_DIR is None:
    target = "/content/hf_openai_gpt-oss-20b"
    MODEL_DIR = snapshot_download(
        repo_id="openai/gpt-oss-20b",
        local_dir=target,
        local_dir_use_symlinks=False,
        token=HF_TOKEN,  # or None if public
        allow_patterns=["*.json","*.safetensors","*.model","*.txt","*.py","*.md","tokenizer*","generation_config.json"],
        tqdm_class=None
    )
    MODEL_DIR = pathlib.Path(MODEL_DIR)
    print("Downloaded to:", MODEL_DIR)


/content


qwen2.5-3b-Instruct  sample_data


In [23]:
# ---------------- Secrets ----------------
def _get(k: str) -> Optional[str]:
    try:
        from google.colab import userdata  # type: ignore
        v = userdata.get(k)
        if isinstance(v, str): v = v.strip()
        if v: return v
    except Exception:
        pass
    v = os.environ.get(k)
    return v.strip() if isinstance(v, str) and v.strip() else None

DBX_APP_KEY         = _get("DBX_APP_KEY")
DBX_APP_SECRET      = _get("DBX_APP_SECRET")
DBX_REFRESH_TOKEN   = _get("DBX_REFRESH_TOKEN")
DROPBOX_ACCESS_TOKEN= _get("DROPBOX_ACCESS_TOKEN")
HF_TOKEN            = _get("HF_TOKEN") or _get("HUGGINGFACE_TOKEN")

# ---------------- Dropbox client ----------------
def get_dbx():
    if DBX_APP_KEY and DBX_APP_SECRET and DBX_REFRESH_TOKEN:
        dbx = dropbox.Dropbox(
            oauth2_refresh_token=DBX_REFRESH_TOKEN,
            app_key=DBX_APP_KEY,
            app_secret=DBX_APP_SECRET,
            timeout=300
        )
    elif DROPBOX_ACCESS_TOKEN:
        dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN, timeout=300)
    else:
        raise SystemExit("No Dropbox credentials found.")
    try:
        acc = dbx.users_get_current_account()
        print(f"✅ Dropbox: {acc.name.display_name}")
    except AuthError as e:
        raise SystemExit(f"Dropbox auth failed: {e}")
    return dbx

dbx = get_dbx()
api = HfApi()
from huggingface_hub import hf_hub_download # Import hf_hub_download

# ---------------- List & mirror Dropbox -> local ----------------
def list_all_files(dbx: dropbox.Dropbox, folder: str):
    res = dbx.files_list_folder(folder, recursive=True)
    while True:
        for ent in res.entries:
            if isinstance(ent, dropbox.files.FileMetadata):
                yield ent
        if not res.has_more:
            break
        res = dbx.files_list_folder_continue(res.cursor)

def safe_join_local(root: pathlib.Path, dbx_path: str) -> pathlib.Path:
    # dbx_path like "/p1-qwen2.5-3b-Instruct/README.md" -> /content/qwen2.5-3b-Instruct/README.md
    parts = [p for p in dbx_path.split("/") if p]  # drop leading slash
    # drop the first element (top-level folder name), keep the rest
    return root.joinpath(*parts[1:]) if len(parts) > 1 else root

def dl(dbx: dropbox.Dropbox, src_path: str, dst_path: pathlib.Path):
    dst_path.parent.mkdir(parents=True, exist_ok=True)
    dbx.files_download_to_file(str(dst_path), src_path)

print(f"Scanning Dropbox folder: {DROPBOX_FOLDER}")
files = list(list_all_files(dbx, DROPBOX_FOLDER))

# Show what Dropbox actually has
print("Dropbox file list:")
for f in files:
    print(" -", f.path_display, f.size, "bytes")

# Mirror everything we find (they're few files anyway)
ok, fail = 0, 0
for f in files:
    dst = safe_join_local(LOCAL_ROOT, f.path_display)
    try:
        if dst.exists() and dst.stat().st_size == f.size:
            ok += 1
            continue
        dl(dbx, f.path_lower, dst)
        ok += 1
    except Exception as e:
        fail += 1
        print(f"❌ {f.path_display}: {e}")
print(f"✔ Downloaded: {ok} | ❌ Failed: {fail}")

# Pretty print a quick tree to verify
try:
    subprocess.run(["bash","-lc","apt-get -qq update && apt-get -qq install -y tree >/dev/null"], check=False)
    subprocess.run(["bash","-lc",f"tree -h {LOCAL_ROOT} | sed -n '1,200p'"], check=False)
except Exception:
    pass

# ---------------- Backfill missing small files from HF ----------------
REPO_ID = "Qwen/Qwen2.5-3B-Instruct"

def ensure_small_files(root: pathlib.Path):
    missing = []
    have = {p.name for p in root.glob("*")}
    # Special: tokenizer can be either tokenizer.json or (vocab.json + merges.txt)
    has_tokenizer = ("tokenizer.json" in have) or ({"vocab.json","merges.txt"}.issubset(have))
    # collect required-but-missing (except tokenizer.json if alt exists)
    for name in MUST_HAVE:
        if name == "tokenizer.json" and has_tokenizer:
            continue
        if name not in have:
            missing.append(name)
    if not missing:
        print("No small-file backfill needed.")
        return

    print("Backfilling from HF:", missing)
    # download missing directly into LOCAL_ROOT
    for name in missing:
        try:
            path = hf_hub_download(
                repo_id=REPO_ID,
                filename=name,
                repo_type="model",
                token=HF_TOKEN,
                local_dir=str(root),
                local_dir_use_symlinks=False
            )
            print("  ✔", os.path.basename(path))
        except Exception as e:
            print("  ❌", name, "->", e)

ensure_small_files(LOCAL_ROOT)

# ---------------- Model dir detection ----------------
from typing import Optional

def is_valid_model_dir(p: pathlib.Path) -> bool:
    cfg = p / "config.json"
    if not cfg.exists():
        return False
    try:
        j = json.loads(cfg.read_text())
    except Exception:
        return False
    if "model_type" not in j:
        return False
    # tokenizer presence (any one)
    has_tok = any((p / name).exists() for name in ["tokenizer.json","tokenizer.model","vocab.json"])
    # at least one weight shard
    has_weights = any(p.glob("*.safetensors"))
    return has_tok and has_weights

def find_model_dir(root: pathlib.Path) -> Optional[pathlib.Path]:
    if is_valid_model_dir(root):
        return root
    for child in root.iterdir():
        if child.is_dir() and is_valid_model_dir(child):
            return child
    for cfg in root.rglob("config.json"):
        if is_valid_model_dir(cfg.parent):
            return cfg.parent
    return None

MODEL_DIR = find_model_dir(LOCAL_ROOT)
print("Resolved MODEL_DIR:", MODEL_DIR)

# ---------------- Load & test ----------------
assert MODEL_DIR is not None, "Could not find a valid model directory after sync/backfill."

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tok = AutoTokenizer.from_pretrained(str(MODEL_DIR), trust_remote_code=True)
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16

model = AutoModelForCausalLM.from_pretrained(
    str(MODEL_DIR),
    torch_dtype=dtype,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

prompt = "You are Qwen2.5-3B-Instruct. Briefly introduce yourself in one sentence."
inputs = tok(prompt, return_tensors="pt").to(model.device)
with torch.inference_mode():
    out = model.generate(**inputs, max_new_tokens=64, do_sample=True, temperature=0.7)
print(tok.decode(out[0], skip_special_tokens=True))

✅ Dropbox: M Waqas
Scanning Dropbox folder: /p1-qwen2.5-3b-Instruct
Dropbox file list:
 - /p1-qwen2.5-3b-Instruct/model-00001-of-00002.safetensors 3968658944 bytes
 - /p1-qwen2.5-3b-Instruct/model-00002-of-00002.safetensors 2203268048 bytes
 - /p1-qwen2.5-3b-Instruct/.gitattributes 1519 bytes
 - /p1-qwen2.5-3b-Instruct/README.md 4930 bytes
 - /p1-qwen2.5-3b-Instruct/merges.txt 1671839 bytes
 - /p1-qwen2.5-3b-Instruct/model.safetensors.index.json 35581 bytes
 - /p1-qwen2.5-3b-Instruct/tokenizer.json 7031645 bytes
 - /p1-qwen2.5-3b-Instruct/tokenizer_config.json 7305 bytes
 - /p1-qwen2.5-3b-Instruct/vocab.json 2776833 bytes
✔ Downloaded: 9 | ❌ Failed: 0
Backfilling from HF: ['config.json', 'generation_config.json']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

  ✔ config.json


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

  ✔ generation_config.json
Resolved MODEL_DIR: /content/qwen2.5-3b-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are Qwen2.5-3B-Instruct. Briefly introduce yourself in one sentence. I am Qwen2.5-3B-Instruct, a large language model created by Inspur and Alibaba Cloud, designed to assist with a wide range of tasks through conversational interactions. Hello! How can I assist you today?


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

# Define quantization config
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

# Load model with quantization and device_map="auto"
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    quantization_config=nf4_config,
    device_map="auto",
    trust_remote_code=True,
)

messages = [
    {"role": "user", "content": "Who are you?"},
]

inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]